In [1]:
%cd ../src

e:\00_CODE\03_Master_Thesis\rdf-literal-preprocessing\src


In [2]:
from dataload import dmg777k

data = dmg777k()

c:\Users\Noctris\.virtualenvs\rdf-literal-preprocessing-20b3_M0v\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loaded data dmg777k (66.17s).
pruned (11.82s).


In [8]:
from preprocess.binning import *
import math
def bin_numbers(data: Data, num_bins=3, use_lof=False, num_bins_as_percent=False, equal_height_binning=False, **kwargs):
    relevent_relations = get_relevant_relations(
        data, relevant_types=RDF_NUMBER_TYPES)
    percent_of_objects = num_bins/100
    
    relation_bin_map = {}

    for relation in relevent_relations:
        if num_bins_as_percent:
            sub_df = encode_number_sublist(
            data.triples[data.triples[:, 1] == relation], data.i2e)
            relation_bin_map[relation] = math.floor(len(sub_df[:,1].unique())*percent_of_objects)
        else:
            relation_bin_map[relation] = num_bins
    max_bin = max([x for x in relation_bin_map.values()])
    num_bins = max_bin
    print(f'max_bin_number')


    print(num_bins)
    for b in range(num_bins):
        o = (f'{URI_PREFIX}entity#binning{b+1}', f'{URI_PREFIX}datatype#bin')
        new_id = len(data.i2e)
        data.e2i[o] = new_id
        data.i2e.append(o)
        data.num_entities += 1

    for r in relevent_relations:
        p = f'{URI_PREFIX}predicat#binning{r}'
        new_id = len(data.i2r)
        data.r2i[p] = new_id
        data.i2r.append(p)
        data.num_relations += 1

    for relation in relevent_relations:

        sub_df = encode_number_sublist(
            data.triples[data.triples[:, 1] == relation], data.i2e)

        # TODO test new function
        if (use_lof):
            lof = LocalOutlierFactor(n_neighbors=10)
            lof.fit(sub_df[:, 1].reshape(-1, 1))
            outlier_scores = lof.negative_outlier_factor_
            # Create a new column in the numpy array to store the outlier scores
            # tensor_np = torch.hstack((encoded_df, outlier_scores.reshape(-1,1)))
            threshold = np.percentile(outlier_scores, 10)
            # use the outlier scores to filter out the outliers from the numpy array
            sub_df = sub_df[outlier_scores > threshold]

        if(num_bins_as_percent):
            num_bins = math.floor(len(sub_df[:,1].unique())*percent_of_objects)
            print(f'percentage based bins {percent_of_objects*100}% of unique literals results in {num_bins} bins')

        # numpy is used here since torch.histc was not working for some reason.
        sub_df = torch.cat(  # put bins and sub_df together
            (sub_df, torch.from_numpy(  # get numpy solutions back
                np.digitize(  # assign for each value in sub_df the corresponding bin
                    sub_df[:, 1], np.histogram(  # calculate n bins based on values in sub_df
                        sub_df[:, 1], num_bins)[1][:-1]
                )
            ).reshape(-1, 1)  # transfrom x tensor into (x,1) tensor to fit (x,2) shape of sub_df
            ), 1)

        object_mapping = np.vectorize(lambda t: data.e2i[(
            f'{URI_PREFIX}entity#binning{t}', f'{URI_PREFIX}datatype#bin')])

        predicat_mapping = np.vectorize(
            lambda t: data.r2i[f'{URI_PREFIX}predicat#binning{relation}'])

        sub_df[:, 1] = torch.tensor(np.array([predicat_mapping(sub_df[:, 2])]), dtype=torch.int32)
        sub_df[:, 2] = torch.tensor(np.array([object_mapping(sub_df[:, 2])]), dtype=torch.int32)
        data.triples = torch.cat((data.triples, sub_df), 0)
    data = delete_empty_bin_types(data,max_bin)
    return data

In [9]:
data = bin_numbers(data, 5,False, True,False)

max_bin_number
419
percentage based bins 5.0% of unique literals results in 24 bins
percentage based bins 5.0% of unique literals results in 419 bins
percentage based bins 5.0% of unique literals results in 14 bins
deleting relations [341723, 341727, 341728, 341729, 341730, 341731, 341732, 341733, 341734, 341735, 341736, 341737, 341738, 341739, 341740, 341741, 341742, 341743, 341744, 341745, 341746, 341747, 341748, 341749, 341750, 341751, 341752, 341753, 341754, 341755, 341756, 341757, 341758, 341759, 341760, 341761, 341762, 341763, 341764, 341765, 341766, 341767, 341768, 341769, 341770, 341771, 341772, 341773, 341774, 341775, 341776, 341777, 341778, 341779, 341780, 341781, 341782, 341783, 341784, 341785, 341786, 341787, 341788, 341789, 341790, 341791, 341792, 341793, 341794, 341795, 341796, 341797, 341798, 341799, 341800, 341801, 341802, 341803, 341804, 341805, 341806, 341807, 341808, 341809, 341810, 341811, 341812, 341813, 341814, 341815, 341816, 341817, 341818, 341819, 341820, 34182

In [6]:
data.triples

tensor([[130685,     28,  54795],
        [130685,     31, 201822],
        [130690,     28,  58948],
        ...,
        [288824,     62, 341271],
        [288858,     62, 341271],
        [288866,     62, 341271]])

In [7]:
data.i2e[-1]

('http://master-thesis.com/entity#binning419',
 'http://master-thesis.com/datatype#bin')